# **Stable Diffusion - LoRA Dreambooth**

# I. Installation

In [5]:
# @title ## 1.1. Install Dependencies
# @markdown Clone Kohya Trainer from GitHub and check for updates. Use textbox below if you want to checkout other branch or old commit. Leave it empty to stay the HEAD on main.  This will also install the required libraries.
import os
import zipfile
import shutil
import time
from subprocess import getoutput
from IPython.utils import capture

# root_dir
root_dir = "./"
deps_dir = os.path.join(root_dir, "deps")
repo_dir = os.path.join(root_dir, "model")
training_dir = os.path.join(root_dir, "train", "LoRA")
pretrained_model = os.path.join(root_dir, "checkpoints", "pretrained_model")
vae_dir = os.path.join(root_dir, "checkpoints", "vae")
config_dir = os.path.join(training_dir, "config")

# repo_dir
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir, "tools")
finetune_dir = os.path.join(repo_dir, "finetune")

# bitsandytes_main_py = "/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py"
install_xformers = False 
mount_drive = False  # @param {type: "boolean"}
verbose = False # @param {type: "boolean"}


# II. Pretrained Models

In [6]:
# @title ## 2.1. Download Available Model
import os

# Get model here: https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/stable_diffusion_1_5-pruned.safetensors

models = {
    "Stable-Diffusion-v1-5": "./checkpoints/pretrained_model/v1-5-pruned-emaonly.safetensors",
}

In [7]:
# @title ## 2.3. Download Available VAE (Optional)
import os

# Download compatible SD v1.5 VAE from "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt" 

vaes = {
    "stablediffusion.vae.pt": "./checkpoints/vae-ft-mse-840000-ema-pruned.ckpt",
}


# III. Data

In [8]:
# Get data from training folder
import os

train_data_dir = "./train/LoRA/train_data"


# Model Training 



In [9]:
# @title ## Model Config
# Change the key to:
# - living
# - bedroom
# - dining
key = "living"
project_name = "roomifai_" + key
if not project_name:
    project_name = "last"
pretrained_model_name_or_path = "./checkpoints/pretrained_model/v1-5-pruned-emaonly.safetensors"
vae = "./checkpoints/checkpoints/vae/vae-ft-mse-840000-ema-pruned.ckpt"
output_dir = "./train/LoRA/output"
output_to_drive = True

sample_dir = os.path.join(output_dir, "sample")
for dir in [output_dir, sample_dir]:
    os.makedirs(dir, exist_ok=True)

print("Project Name: ", project_name)
print(
    "Pretrained Model Path: ", pretrained_model_name_or_path
) if pretrained_model_name_or_path else print("No pretrained model specified.")
print("VAE path: ", vae) if vae else print("No VAE path specified.")
print("Output path: ", output_dir)

Project Name:  roomifai_living
Pretrained Model Path:  ./checkpoints/pretrained_model/v1-5-pruned-emaonly.safetensors
VAE path:  ./checkpoints/checkpoints/vae/vae-ft-mse-840000-ema-pruned.ckpt
Output path:  ./train/LoRA/output


In [10]:
# @title ## 5.2. Dataset Config
import os
import toml
import glob

dataset_repeats = 1
# `activation_word` is not used in training if you train with captions, but it is still printed to metadata.
activation_word = "roomifai"
caption_extension = ".txt"
resolution = 512
flip_aug = True
keep_tokens = 0

# Place the images and captions under the training directory
# Name it as {number of steps}_{class}
# e.g. 10_living

def parse_folder_name(folder_name, default_num_repeats, default_class_token):
    folder_name_parts = folder_name.split("_")

    if len(folder_name_parts) == 2:
        if folder_name_parts[0].isdigit():
            num_repeats = int(folder_name_parts[0])
            class_token = folder_name_parts[1].replace("_", " ")
        else:
            num_repeats = default_num_repeats
            class_token = default_class_token
    else:
        num_repeats = default_num_repeats
        class_token = default_class_token

    return num_repeats, class_token

def find_image_files(path):
    supported_extensions = (".png", ".jpg", ".jpeg", ".webp", ".bmp")
    return [file for file in glob.glob(path + '/**/*', recursive=True) if file.lower().endswith(supported_extensions)]

def process_data_dir(data_dir, default_num_repeats, default_class_token, is_reg=False):
    subsets = []

    images = find_image_files(data_dir)
    if images:
        subsets.append({
            "image_dir": data_dir,
            "class_tokens": default_class_token,
            "num_repeats": default_num_repeats,
            **({"is_reg": is_reg} if is_reg else {}),
        })

    for root, dirs, files in os.walk(data_dir):
        for folder in dirs:
            folder_path = os.path.join(root, folder)
            images = find_image_files(folder_path)

            if images:
                num_repeats, class_token = parse_folder_name(folder, default_num_repeats, default_class_token)

                subset = {
                    "image_dir": folder_path,
                    "class_tokens": class_token,
                    "num_repeats": num_repeats,
                }

                if is_reg:
                    subset["is_reg"] = True

                subsets.append(subset)

    return subsets


train_subsets = process_data_dir(train_data_dir, dataset_repeats, activation_word)

subsets = train_subsets

config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": caption_extension,
        "shuffle_caption": True,
        "keep_tokens": keep_tokens,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": resolution,
            "min_bucket_reso": 320 if resolution > 640 else 256,
            "max_bucket_reso": 1280 if resolution > 640 else 1024,
            "caption_dropout_rate": 0,
            "caption_tag_dropout_rate": 0,
            "caption_dropout_every_n_epochs": 0,
            "flip_aug": flip_aug,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": subsets,
        }
    ],
}

dataset_config = os.path.join(config_dir, "dataset_config.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(dataset_config, "w") as f:
    f.write(config_str)

print(config_str)

[[datasets]]
resolution = 512
min_bucket_reso = 256
max_bucket_reso = 1024
caption_dropout_rate = 0
caption_tag_dropout_rate = 0
caption_dropout_every_n_epochs = 0
flip_aug = true
color_aug = false
[[datasets.subsets]]
image_dir = "./train/LoRA/train_data"
class_tokens = "roomifai"
num_repeats = 1

[[datasets.subsets]]
image_dir = "./train/LoRA/train_data\\10_living"
class_tokens = "living"
num_repeats = 10


[general]
enable_bucket = true
caption_extension = ".txt"
shuffle_caption = true
keep_tokens = 0
bucket_reso_steps = 64
bucket_no_upscale = false



In [11]:
# @title ## 5.3. LoRA and Optimizer Config

# @markdown ### LoRA Config:
network_category = "LoRA"
conv_dim = 32
conv_alpha = 16
network_dim = 32
network_alpha = 16
# For resume training.
network_weight = ""
network_module = "networks.lora"
network_args = "" if network_category == "LoRA" else [
    f"conv_dim={conv_dim}", f"conv_alpha={conv_alpha}",
    ]

# @markdown ### <br>Optimizer Config:
# `NEW` Gamma for reducing the weight of high-loss timesteps. Lower numbers have a stronger effect. The paper recommends 5. Read the paper [here](https://arxiv.org/abs/2303.09556).
min_snr_gamma = -1
optimizer_type = "AdamW8bit"
optimizer_args = ""
train_unet = True 
unet_lr = 1e-4
train_text_encoder = True 
text_encoder_lr = 5e-5
lr_scheduler = "constant"
lr_warmup_steps = 0
lr_scheduler_num_cycles = 0
lr_scheduler_power = 0

if network_category == "LoHa":
  network_args.append("algo=loha")
elif network_category == "LoCon_Lycoris":
  network_args.append("algo=lora")

print("- LoRA Config:")
print(f"  - Min-SNR Weighting: {min_snr_gamma}") if not min_snr_gamma == -1 else ""
print(f"  - Loading network module: {network_module}")
if not network_category == "LoRA":
  print(f"  - network args: {network_args}")
print(f"  - {network_module} linear_dim set to: {network_dim}")
print(f"  - {network_module} linear_alpha set to: {network_alpha}")
if not network_category == "LoRA":
  print(f"  - {network_module} conv_dim set to: {conv_dim}")
  print(f"  - {network_module} conv_alpha set to: {conv_alpha}")

if not network_weight:
    print("  - No LoRA weight loaded.")
else:
    if os.path.exists(network_weight):
        print(f"  - Loading LoRA weight: {network_weight}")
    else:
        print(f"  - {network_weight} does not exist.")
        network_weight = ""

print("- Optimizer Config:")
print(f"  - Additional network category: {network_category}")
print(f"  - Using {optimizer_type} as Optimizer")
if optimizer_args:
    print(f"  - Optimizer Args: {optimizer_args}")
if train_unet and train_text_encoder:
    print("  - Train UNet and Text Encoder")
    print(f"    - UNet learning rate: {unet_lr}")
    print(f"    - Text encoder learning rate: {text_encoder_lr}")
if train_unet and not train_text_encoder:
    print("  - Train UNet only")
    print(f"    - UNet learning rate: {unet_lr}")
if train_text_encoder and not train_unet:
    print("  - Train Text Encoder only")
    print(f"    - Text encoder learning rate: {text_encoder_lr}")
print(f"  - Learning rate warmup steps: {lr_warmup_steps}")
print(f"  - Learning rate Scheduler: {lr_scheduler}")
if lr_scheduler == "cosine_with_restarts":
    print(f"  - lr_scheduler_num_cycles: {lr_scheduler_num_cycles}")
elif lr_scheduler == "polynomial":
    print(f"  - lr_scheduler_power: {lr_scheduler_power}")


- LoRA Config:
  - Loading network module: networks.lora
  - networks.lora linear_dim set to: 32
  - networks.lora linear_alpha set to: 16
  - No LoRA weight loaded.
- Optimizer Config:
  - Additional network category: LoRA
  - Using AdamW8bit as Optimizer
  - Train UNet and Text Encoder
    - UNet learning rate: 0.0001
    - Text encoder learning rate: 5e-05
  - Learning rate warmup steps: 0
  - Learning rate Scheduler: constant


In [12]:
# @title ## 5.4. Training Config

import toml
import os

lowram = True
enable_sample_prompt = True
sampler = "euler_a"
noise_offset = 0.0
num_epochs = 4
vae_batch_size = 4
train_batch_size = 6
mixed_precision = "bf16"
save_precision = "bf16"
save_n_epochs_type = "save_every_n_epochs"  # @param ["save_every_n_epochs", "save_n_epoch_ratio"]
save_n_epochs_type_value = 1
save_model_as = "safetensors"
max_token_length = 225
clip_skip = 2
gradient_checkpointing = False
gradient_accumulation_steps = 1
seed = -1
logging_dir = "./train/LoRA/logs"
prior_loss_weight = 1.0

# os.chdir(repo_dir)

sample_str = f"""
  masterpiece, best quality, living room with minimalist modern wooden furniture \\
  --w 512 \
  --h 512 \
  --l 7 \
  --s 28    
"""

config = {
    "model_arguments": {
        "v2": False,
        "v_parameterization": False,
        "pretrained_model_name_or_path": pretrained_model_name_or_path,
        "vae": vae,
    },
    "additional_network_arguments": {
        "no_metadata": False,
        "unet_lr": float(unet_lr) if train_unet else None,
        "text_encoder_lr": float(text_encoder_lr) if train_text_encoder else None,
        "network_weights": network_weight,
        "network_module": network_module,
        "network_dim": network_dim,
        "network_alpha": network_alpha,
        "network_args": network_args,
        "network_train_unet_only": True if train_unet and not train_text_encoder else False,
        "network_train_text_encoder_only": True if train_text_encoder and not train_unet else False,
        "training_comment": None,
    },
    "optimizer_arguments": {
        "min_snr_gamma": min_snr_gamma if not min_snr_gamma == -1 else None,
        "optimizer_type": optimizer_type,
        "learning_rate": unet_lr,
        "max_grad_norm": 1.0,
        "optimizer_args": eval(optimizer_args) if optimizer_args else None,
        "lr_scheduler": lr_scheduler,
        "lr_warmup_steps": lr_warmup_steps,
        "lr_scheduler_num_cycles": lr_scheduler_num_cycles if lr_scheduler == "cosine_with_restarts" else None,
        "lr_scheduler_power": lr_scheduler_power if lr_scheduler == "polynomial" else None,
    },
    "dataset_arguments": {
        "cache_latents": True,
        "debug_dataset": False,
        "vae_batch_size": vae_batch_size,
    },
    "training_arguments": {
        "output_dir": output_dir,
        "output_name": project_name,
        "save_precision": save_precision,
        "save_every_n_epochs": save_n_epochs_type_value if save_n_epochs_type == "save_every_n_epochs" else None,
        "save_n_epoch_ratio": save_n_epochs_type_value if save_n_epochs_type == "save_n_epoch_ratio" else None,
        "save_last_n_epochs": None,
        "save_state": None,
        "save_last_n_epochs_state": None,
        "resume": None,
        "train_batch_size": train_batch_size,
        "max_token_length": 225,
        "mem_eff_attn": False,
        "xformers": True,
        "max_train_epochs": num_epochs,
        "max_data_loader_n_workers": 8,
        "persistent_data_loader_workers": True,
        "seed": seed if seed > 0 else None,
        "gradient_checkpointing": gradient_checkpointing,
        "gradient_accumulation_steps": gradient_accumulation_steps,
        "mixed_precision": mixed_precision,
        "clip_skip": clip_skip,
        "project_dir": logging_dir,
        "log_prefix": project_name,
        "noise_offset": noise_offset if noise_offset > 0 else None,
        "lowram": lowram,
    },
    "sample_prompt_arguments": {
        "sample_every_n_steps": None,
        "sample_every_n_epochs": 1 if enable_sample_prompt else 999999,
        "sample_sampler": sampler,
    },
    "dreambooth_arguments": {
        "prior_loss_weight": 1.0,
    },
    "saving_arguments": {
        "save_model_as": save_model_as
    },
}

config_path = os.path.join(config_dir, "config_file.toml")
prompt_path = os.path.join(config_dir, "sample_prompt.txt")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

write_file(config_path, config_str)
write_file(prompt_path, sample_str)
    
print(config_str)

[model_arguments]
v2 = false
v_parameterization = false
pretrained_model_name_or_path = "./checkpoints/pretrained_model/v1-5-pruned-emaonly.safetensors"
vae = "./checkpoints/checkpoints/vae/vae-ft-mse-840000-ema-pruned.ckpt"

[additional_network_arguments]
no_metadata = false
unet_lr = 0.0001
text_encoder_lr = 5e-5
network_module = "networks.lora"
network_dim = 32
network_alpha = 16
network_train_unet_only = false
network_train_text_encoder_only = false

[optimizer_arguments]
optimizer_type = "AdamW8bit"
learning_rate = 0.0001
max_grad_norm = 1.0
lr_scheduler = "constant"
lr_warmup_steps = 0

[dataset_arguments]
cache_latents = true
debug_dataset = false
vae_batch_size = 4

[training_arguments]
output_dir = "./train/LoRA/output"
output_name = "roomifai_living"
save_precision = "bf16"
save_every_n_epochs = 1
train_batch_size = 6
max_token_length = 225
mem_eff_attn = false
xformers = true
max_train_epochs = 4
max_data_loader_n_workers = 8
persistent_data_loader_workers = true
gradient_ch

In [13]:
#@title ## 5.5. Start Training
sample_prompt = os.path.join(config_dir, "sample_prompt.txt")
config_file = os.path.join(config_dir, "config_file.toml")
dataset_config = os.path.join(config_dir, "dataset_config.toml")

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts" : sample_prompt,
    "dataset_config" : dataset_config,
    "config_file" : config_file
}

def train(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

accelerate_args = train(accelerate_conf)
train_args = train(train_conf)
final_args = f"accelerate launch {accelerate_args} train_network.py {train_args}"

print("Running: ", final_args)

# os.chdir(repo_dir)
!{final_args}

Running:  accelerate launch --config_file="./model\accelerate_config/config.yaml" --num_cpu_threads_per_process=1  train_network.py --sample_prompts="./train\LoRA\config\sample_prompt.txt" --dataset_config="./train\LoRA\config\dataset_config.toml" --config_file="./train\LoRA\config\config_file.toml" 
Loading settings from ./train\LoRA\config\config_file.toml...
./train\LoRA\config\config_file
prepare tokenizer
update token length: 225
Loading dataset config from train\LoRA\config\dataset_config.toml
prepare images.
found directory ./train/LoRA/train_data contains 0 image files
ignore subset with image_dir='./train/LoRA/train_data': no images found / 画像が見つからないためサブセットを無視します
found directory ./train/LoRA/train_data\10_living contains 160 image files
1600 train images with repeating.
0 reg images.
no regularization images / 正則化画像が見つかりませんでした
[Dataset 0]
  batch_size: 6
  resolution: (512, 512)
  enable_bucket: True
  min_bucket_reso: 256
  max_bucket_reso: 1024
  bucket_reso_steps: 64
  buck


100%|██████████| 160/160 [00:00<00:00, 1713.04it/s]
c:\users\jackchua\miniconda3\envs\room\lib\site-packages\safetensors\torch.py:98: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  with safe_open(filename, framework="pt", device=device) as f:
Traceback (most recent call last):
  File "c:\users\jackchua\miniconda3\envs\room\lib\site-packages\diffusers\models\modeling_utils.py", line 857, in _get_model_file
    model_file = hf_hub_download(
  File "c:\users\jackchua\miniconda3\envs\room\lib\site-packages\huggingface_hub\utils\_validators.py", line 110, in _inner_fn
    validate_repo_id(arg_value)
  File "c:\users\jackchua\miniconda3\envs\room\lib\site-packages\huggingface_hub\utils\_validators.py", line 158, in validate_repo_id
    raise HFValidationE

In [14]:
# @title ## 6.2. Interrogating LoRA Weights
# @markdown Now you can check if your LoRA trained properly.
import os
import torch
import json
from safetensors.torch import load_file
from safetensors.torch import safe_open

# @markdown If you used `clip_skip = 2` during training, the values of `lora_te_text_model_encoder_layers_11_*` will be `0.0`, this is normal. These layers are not trained at this value of `Clip Skip`.
network_weight = ""
verbose = False

def is_safetensors(path):
    return os.path.splitext(path)[1].lower() == ".safetensors"

def load_weight_data(file_path):
    if is_safetensors(file_path):
        return load_file(file_path)
    else:
        return torch.load(file_path, map_location="cuda")

def extract_lora_weights(weight_data):
    lora_weights = [
        (key, weight_data[key])
        for key in weight_data.keys()
        if "lora_up" in key or "lora_down" in key
    ]
    return lora_weights

def print_lora_weight_stats(lora_weights):
    print(f"Number of LoRA modules: {len(lora_weights)}")

    for key, value in lora_weights:
        value = value.to(torch.float32)
        print(f"{key}, {torch.mean(torch.abs(value))}, {torch.min(torch.abs(value))}")

def print_metadata(file_path):
    if is_safetensors(file_path):
        with safe_open(file_path, framework="pt") as f:
            metadata = f.metadata()
        if metadata is not None:
            print(f"\nLoad metadata for: {file_path}")
            print(json.dumps(metadata, indent=4))
    else:
        print("No metadata saved, your model is not in safetensors format")

def main(file_path, verbose: bool):
    weight_data = load_weight_data(file_path)

    if verbose:
        lora_weights = extract_lora_weights(weight_data)
        print_lora_weight_stats(lora_weights)

    print_metadata(file_path)

if __name__ == "__main__":
    main(network_weight, verbose)

FileNotFoundError: [Errno 2] No such file or directory: ''

In [ ]:
# @title ## 6.3. Inference
%store -r

# @markdown ### LoRA Config
# @markdown Currently, `LoHa` and `LoCon_Lycoris` are not supported. Please run `Portable Web UI` instead
network_weight = ""
network_mul = 0.7  # @param {type:"slider", min:-1, max:2, step:0.05}
network_module = "networks.lora"
network_args = ""

# @markdown ### <br> General Config
v2 = False
v_parameterization = False
prompt = "masterpiece, best quality, 1girl, aqua eyes, baseball cap, blonde hair, closed mouth, earrings, green background, hat, hoop earrings, jewelry, looking at viewer, shirt, short hair, simple background, solo, upper body, yellow shirt"  # @param {type: "string"}
negative = "lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry"  # @param {type: "string"}
model = "/content/pretrained_model/AnyLoRA.safetensors"  # @param {type: "string"}
vae = ""  # @param {type: "string"}
outdir = "/content/tmp"  # @param {type: "string"}
scale = 7  # @param {type: "slider", min: 1, max: 40}
sampler = "ddim"  # @param ["ddim", "pndm", "lms", "euler", "euler_a", "heun", "dpm_2", "dpm_2_a", "dpmsolver","dpmsolver++", "dpmsingle", "k_lms", "k_euler", "k_euler_a", "k_dpm_2", "k_dpm_2_a"]
steps = 28  # @param {type: "slider", min: 1, max: 100}
precision = "fp16"  # @param ["fp16", "bf16"] {allow-input: false}
width = 512  # @param {type: "integer"}
height = 768  # @param {type: "integer"}
images_per_prompt = 4  # @param {type: "integer"}
batch_size = 4  # @param {type: "integer"}
clip_skip = 2  # @param {type: "slider", min: 1, max: 40}
seed = -1  # @param {type: "integer"}

final_prompt = f"{prompt} --n {negative}"

config = {
    "v2": v2,
    "v_parameterization": v_parameterization,
    "network_module": network_module,
    "network_weight": network_weight,
    "network_mul": float(network_mul),
    "network_args": eval(network_args) if network_args else None,
    "ckpt": model,
    "outdir": outdir,
    "xformers": True,
    "vae": vae if vae else None,
    "fp16": True,
    "W": width,
    "H": height,
    "seed": seed if seed > 0 else None,
    "scale": scale,
    "sampler": sampler,
    "steps": steps,
    "max_embeddings_multiples": 3,
    "batch_size": batch_size,
    "images_per_prompt": images_per_prompt,
    "clip_skip": clip_skip if not v2 else None,
    "prompt": final_prompt,
}

args = ""
for k, v in config.items():
    if k.startswith("_"):
        args += f'"{v}" '
    elif isinstance(v, str):
        args += f'--{k}="{v}" '
    elif isinstance(v, bool) and v:
        args += f"--{k} "
    elif isinstance(v, float) and not isinstance(v, bool):
        args += f"--{k}={v} "
    elif isinstance(v, int) and not isinstance(v, bool):
        args += f"--{k}={v} "

final_args = f"python gen_img_diffusers.py {args}"

# os.chdir(repo_dir)
!{final_args}